# 1 - Churn = Happiness: How To Keep Customers Happy with Analytics
## Part 1: A Churn Prediction Use-Case for KKBox Music using Google BigQuery, Apache Spark, and XGBoost

“There is only one boss. The customer. And he can fire everybody in the company from the chairman on down, simply by spending his money somewhere else.” - Sam Walton

Churn is one of the least favorite words for a business owner as it means that they have just lost a customer for good. According to Forrester, "it costs 5 times more to acquire a customer than to retain and enrich customers, ***but the majority of companies are failing to make the right kind of loyalty investment.***" And although this is an *inevitable* part of doing business, many of these customers who churn can be retained if companies implement well-timed marketing and sales approaches. The art of this timing, however, comes down to how we answer one question: "Why and When did my customer become unhappy?"

If you were to ask any Data Scientist to tell you what the purpose of a Churn Prediction Model is, they would most likely answer with "to predict churn... duh". That answer, surprisingly, is correct. However, it only goes halfway into solving the real problem. Let's say we have a customer, her name is Karen. It was determined that Karen has a 73% chance of churning next month. This is as far as a standard Churn Prediction project will get us but it leaves us with the bigger question of *'What do we do now?'*. Should we just bombard Karen with emails and offers to entice her to come back? Better yet, how do we go about keeping Karen as a customer even if everything is running smoothly?

It is only when we become empathetic towards our existing customer base that we start to care about their identity, appreciate how long they have been a loyal customer to us, and understand what we can do to make their experience with us a Happy one. This is how we can take a simple Churn Prediction Model and turn it into a Customer Retention Support System (CRSS). To do this properly we will need to answer the following questions:

**1) Which Customers Have The Highest Probability Of Churning Soon?**
As already stated, a prediction model can be trained to identify the customers that are most likely to leave soon. Although many established organizations have a team dedicated to identifying these potential churns and retaining them, it is likely that through machine learning we can seamlessly evaluate a larger amount of variables and do a better job at identifying these potential customers. With such a tool this team could then solely focus on retention strategies.

**2) What Is The Lifetime Value for Each Of Our Customers?**
Now that we know which customers are high risk, we come across the question of "What incentives do we offer them to reconsider?" (This offer is commonly known as a treatment). At the end of the day, no business wants to lose customers, but they also do not want to overspend to keep them. It is, therefore, crucial to determine the overall value that the company is likely to receive from a particular customer over that person's entire customer lifecycle (i.e. Customer Lifetime Value). By knowing what a customer is worth to the business in the long term, we can begin to approach them with the *right* treatment to get them to stay.

**3) What Retention Offer Would A Particular Customer Best Respond To?** 
Based on the Customer's Lifetime Value, the business can come up with a variety of treatments. After distributing them across high-risk customers, a model can be created to understand which treatments were the most effective. The goal of such a model is one that continuously builds on itself over time and thus the focus of this question turns to the long term.

Throughout 2-3 articles, I will explore these questions using data from an existing business as an example. Each article will follow the CRISP-DM methodology: 
- ***Business Understanding:*** this stage delves into what we want to accomplish from a business perspective
- ***Data Understanding:*** this stage involves collecting, describing, and exploring data
- ***Data Preparation:*** this stage involves cleaning, constructing, and integrating data
- ***Modeling:*** this stage involves building and assessing models
- ***Evaluation:*** in this stage, we assess the degree to which the model meets our business objectives and seek to determine if there is some business reason why this model is deficient
- ***Deployment:*** in this stage, we determine a strategy for deploying results, monitoring the performance of the deployment, and maintaining the deployment

So without further ado, we begin right at Question #1: 

#### <font color=darkgreen>Which Customers Have The Highest Probability Of Churning In The Near Future?</font> CENTER THIS

Retailers Say Customer Retention Will be Key Revenue Growth Driver
https://go.forrester.com/what-it-means/ep04-rethinking-customer-loyalty/

# 1. Business Understanding: Enter KKBox Music

**<font color=orange>#1 KKBoc Logo or Image</font>**

When looking for a dataset for this project I wanted to find something that was massive, dirty, and from a real business. And although today we have access to a plethora of 'toy' datasets, it was really important for me to find something that was exhaustive so I could build multiple projects off of. Coming across KKBOX, was simply bliss.

KKBOX is Asia’s leading music streaming service, holding the world’s most comprehensive Asia-Pop music library with over 30 million tracks. They offer a generous, unlimited version of their service to millions of people, supported by advertising and paid subscriptions. This delicate business model, however, is dependent on accurately predicting churn of their paid users and this is what I hope to deliver on. With such a model we would also, and perhaps more importantly, be able to help explain the main drivers behind customer churn.

# 2. Data Understanding
- ***Data Understanding:*** this stage involves collecting, describing, and exploring data

### 2.1 Dataset Descriptions

The datasets that we will be using for this project are divided into two versions: ***v1*** and ***v2***. We will only be using ***v1*** files as since they contain nearly 3 years worth of data and ***v2*** only contains a single month.
- **train_v1:** containing unique user ids and whether they churned as of ***2/28/2017***. 
- **transactions_v1:** transactions of users up until ***2/28/2017***. It contains features related to plan pricing, transaction dates, and membership expiration dates.
- **user_logs_v1:** daily subscriber activity up until ***2/28/2017***. It contains features related to the number of unique songs played, total seconds listened to per day, and the number of songs partially listened to.
- **members:** All user information data. It contains features related to gender, registration method, city, and age.

#### - <font color=blue>Dataset Statistics</font> -

- ***train_v1 Dataset:*** ~800K Records @ 45.56 MB 
- ***transactions_v1 Dataset:*** ~22 Million Records @ 1.68 GB         
- ***user_log_v1 Dataset:*** ~400 Million Records @ 29.78 GB     
- ***members_v3 Dataset:*** ~5 Million Records @ 417.89 MB
- **<font color=red>Total:  31.92 GB</font>**  w/ 22 raw data points across all files, including 4 date fields

**<font color=orange>#2 ADD A GRAPHIC WITH ALL TABLES AND THEIR RESPECTIVE FEATURES</font>**

Each of the datasets will be imported into Google BigQuery as Raw *RAW_* Tables. These Raw Tables will always remain untouched and will be used as a clean backup if we were to make a mistake in any of our transformations in the future. We will then import these datasets again but as Working Tables. These Working Tables are what we will use to perform all cleaning and data prep as they will provide a safe working environment away from the source datasets. When we get into the feature engineering phase of the project, we will then create a third type of dataset called Derived Tables. These tables will be used for all modeling purposes. I find that creating these 3 different table types allows for a clean and organized workflow. To summarize:
- **Raw Tables (*RAW_*)** - Raw untouched verison of all tables
- **Working Tables (*WRK_*)** - Cleaned and properly formatted version of all tables. These will serve as the source for our Derived Tables.
- **Derived Tables (*DRV_*)** - Table created specifically for our use case's model. All feature engineering will be performed here.

### 2.2 Notes & Observations

Given such a large dataset, I decided it would be best to work with a *persistent* storage solution as manipulating so much data in memory in a notebook was not an option. I originally attempted this using SQLite (too lazy at the time to create a server), but the performance just on my laptop just wasn't enough for me. After realizing that cloud-based Database Management System DBMS was my best option I decided to go with Google BigQuery for this project. This would allow me to seamlessly handle the size of this dataset, and the large amount of feature engineering I would inevitably be performing. Small Disclaimer: I know Google BigQuery isn't intended to be used as a DBMS, and something like a Postgre and MySQL Server would be more appropriate for this use case, but... they offered to let me use it for free!

As for modeling, I used this project as an excuse to use Apache Spark. I have long been fond of distributed computing and with Google also offering customized cluster environments there was simply no time like the present! 

**<font color=orange>#3 ADD FOCUS FACTOR MEME</font>**

# 3. Data Preparation
Please refer to the *Create Derived Tables and Features* section in the *KKBox Churn - Part 1 - ETL* notebook

### 3.1 Derived Table Construction 

As previously stated, the Derived Tables are what we will build our model off of. These tables will be the result of a consolidation of all Working Tables into one single file for each month that is a part of our analysis. We will be using three months (January 2016, February 2016, and March 2016) as Training, Validation, and Testing sets respectively. To begin the construction of these tables let's define the business problem a little more in-depth.

KKBOX defines churn as "***no new valid service subscription within 30 days after the current membership expires***". Although churn values are provided for members in the train_v1 dataset, they are the values for all members taken as of February 28, 2017. As we will be evaluating churn over several previous months we won't be able to use this churn data as it holds "future" information, thus we will need to calculate this ourselves. To do this first we will start by building our monthly Derived Tables with all members who have an <font color=green>*Membership Expiration Date*</font> that lay within each of the respected months. On top of this, we will also include all member-specific information from all tables as well as some simply derived features. Lastly we calculate <font color=green>*is_churn*</font> with respect to the definition. The result:
- membership_expire_date
- payment_method_id
- payment_plan_days
- plan_list_price
- net_paid_amount
- is_net_paid_amount
- is_auto_renew
- is_cancel
- city
- bd
- registered_via
- registration_init_time
- membership_length

**<font color=orange>Add base table Bigquery snapshot</font>**

This table will serve as our base for where we will add our new features onto. As all of our source tables capture different types of user behaviors over different timeframes we can't just join them all together immediately. However, as they do capture behavior over time I believe that there would be a great value if we can get creative on how we capture this ***retrospective data***. We will create these new features through additional queries and python commands.

### 3.3 Feature Engineering

For this project I ended up creating ~235 features, with each of them taking one of two forms; Aggregate and Retrospective Data. 

***The Aggregate Features*** were general aggregations of member data that were for the most part absolute summations, averages, and comparisons. These features were meant to set a static overall** picture of a member's behavior on the application up until this point in time. Examples of these Aggregate Features:
- **current_method_id**: What is the member's current payment plan?
- **current_plan_list_price**: how much does the member pay
- **net_paid_amount**: How does the much member over/under pay?
- **never_active_subscriber**: Binary for Member's who pay each month but have never used the platform
- **total_secs**: Total songs played over lifetime
- **spent_per_num_unq**: Total Spent / Sum of Number of Unique Songs
- **total_cancelations**: Total Amount of Cancelations in Customer Lifetime

***The Retrospective Features*** take three main forms:
- Aggregate Data over Time Intervals of 7, 15, 30, 60, and 120 Days. Examples:
    - **songs_last_7**: How many songs listened to within 7 days of membership_expire_date
    - **total_secs_last_15_AVG**: How many seconds of music listened toon average within 15 days of membership_expire_date
    - **over_50perc_last_30**: How many songs were listend to over 50% within 30 days of membership_expire_date
    - **num_unq_last_60_AVG**: How many unique songs listened to on average within 60 days of membership_expire_date
    
    
- Bi-Weekly Activity Blocks. Examples:
    - **SUM_unq_songs_0_15**: SUM Unique Songs within interval of 0 to 15 days from membership expiration date
    - **AVG_songs_15_30**: AVG Songs within interval of 15 to 30 days from membership expiration date
    - **STD_songs50_30_60**: STD Songs Played Under 50% within interval of 30 to 60 days from membership expiration date
    
    
- Comparison of Bi-Weekly Activity Blocks. Examples:
    - **DIFSUM_unq_songs_0_15_15_30**: SUM Unique Songs within intervals 0-15 and 15-30
    - **DIFAVG_songs_15_30_30_45**: AVG Songs within intervals 15-30 and 30-45
    - **DIFSTD_repeats_30_45_45_60**: STD Repeat Songs Played within intervals 30-45 and 45-60
    


### 3.4 Exploratory Data Analysis - Janurary 2016 (Train Set)
Please refer to the *Create Derived Tables and Features* section in the *KKBox Churn - Part 2 - EDA* notebook

To keep this report concise (EDA is it's own animal), I will be glancing over some of the visual and statistical aspects that were interesting. Specifically, we will look at the Target Variable under center against notable Categorical and Continuous features. As you will see in the notebook I like to take a statistical approach to guide the EDA process. This kind of approach is very efficient when you are trying to analyze a large number of features. For this project I approached this as follows:

**1) Split data into Continuous and Categorical Features**

**2) Compare Continuous Features against our Target Variable**
- Continuous Feature Univariate Analysis
- Independent T-Tests: Continuous Feature Values and their respective Churn Categorical Values
- Box Plots and 5 Number Summaries: 'Statistically Insignificant' Continuous Features vs Churn
- Box Plots and 5 Number Summaries: 'Statistically Significant' Continuous Features vs Churn

**3) Compare Categorical Features against our Target Variable**
- Categorical Feature Univariate Analysis via Frequency Tables
- Chi-Square Tests of Independence: Categorical Feature Values and their respective Churn Categorical Values
- Count Plots: Categorical Features vs Churn

I would encourage you to refer to the *KKBox Churn - Part 2 - EDA* notebook for this project as it is a bit more thorough and goes more in-depth on the above. So without further ado, let's look at January 2016 (Train Set). 

**<font color=orange>Add base Churn Distribution chart</font>**

As you can see we have a severe class imbalance with our Target Variable. Out of 691,899 members in January: 672,751 stayed and 19,148 churned. This Non-Churn to Churn ratio comes out to roughly 97:3. We will need to address this before we get into Modeling, but for now, we will create a 50/50 sub-sample of this dataset to get a better understanding of the behavior differences of Churn vs Non-Churn.

#### <font color=blue>Categorical Variables and Chi-Square Tests of Independece</font>


When analyzing Categorical Features, we can compare their respective contingency table values for Churn @ 1 and 0 to test if they are of the same distribution. Here I am looking to understand if user churn affects the probability of being part of a specific category. This will give us a better understanding of Categorical Feature behavior and their potential influence on our model.

**<font color=orange>Add chi2 results</font>**

With such low P-Values, we can see that all of our Categorical Features are statistically significant. Let's take a deeper look into a couple of them.

**<font color=orange>Add registered/auto plots results</font>**

Looking at both plots you can see the differences in behavior based on Churn. members who have are not setup for auto-recurring payments are more likely to churn than those who are. We can also see that the way a user registers has a significant influence on whether they are likely to churn or not. This last one is quite interesting but unfortunately, KKBOX did not provide any information behind these categorical values.

**<font color=orange>Add total spent and net paid amounts</font>**

These two where by far the most interesting. ***total_spent_zero*** is a binary feature I created to identify members who haven't spent anything in their lifetime and thus have used the platform for free. With the same idea in mind, ***is_net_paid_amount*** was created to state whether a user underpays, overpays, or pays the plan price. When we look at both of these tables we can see that users who do not pay anything or who received a discount are more likely to churn than those who play the plan price. Incredible.

#### <font color=blue>Continuous Features Variables and Independent T-Tests</font>


Moving on to our Continuous Variable analysis, we will start by performing an Independent T-Test of all Continuous Features by splitting their values on our target's categorical values, and then running the test against themselves. Similarly to our Categorical analysis, I am looking to understand if there is a significant difference within these Features when Churn is 0 and 1.

**<font color=orange>Add Ind T-test results</font>**

Out of roughly 230 features created we have 15 features that were deemed insignificant (Level of Significance = .05). We will take a look are both significant and insignificant distributions through box plots.

**<font color=orange>Add non-sig feats</font>**

This is a great illustration of what the distributions for non-significant features look like. For both of these features, each of their respective distributions are nearly identical. Both their bodies and long tails are indistinguishable from each other. With barely any variation, I do not expect these features to contribute to our model.

**<font color=orange>Add sig feats</font>**

Here we have a great example of two features that are both significant. ***total_logins*** shows a clear distinction between churn vs non-churn distributions, showing how non-churn users have more total logins than churned users. ***songs_last_30*** initially can come off as subtle but if we look closely at the main bodies of the distributions and the length and density of the tails we can see that both distributions are quite different. I expect both of these to contribute to our model.

### 3.5 Handling Class Imbalance

As pointed out in the beginning of our Exploratory Data Analysis, we are dealing with a quite severe class imbalance in our Target Variable. With a proportion of 97:3, Non-churn vs churn, we will need to train our model on a balanced set to avoid any dominance by our Non-Churn data. We will start modeling on a 50:50 balanced set and adjust this balance if needed.

# 4. Modeling
Please refer to the *KKBox Churn - Part 3 - Modeling* notebook

There were many models built for this project with many of the features being created as a result of these iterations. We will cover the general evolutions that were undertaken to get to our final model. Our modeling sets were as follows:
- **Train Set:**       January 2016
- **Validation Set**:  February 2016
- **Test Set:**        March 2016

We will be working with two classification estimators: ***Gradient Boosted Trees (GBT)*** and ***Random Forest Classifier (RFC)***. Aside from strong validation scores, we want to find out which model generalizes best.

### Model #1: 1-to-1 Churn vs Non-Churn Ratio Split

We start by modeling on a 1-to-1, Churn vs Non-Churn ratioed subset of our Train data. As we are dealing with a significantly imbalanced dataset we need to train on such a subset to prevent our model from becoming overly biased towards our dominant class.

- **<font color=orange>Model #1: 1-to-1 Ratio Validation CMs</font>**
- **<font color=orange>Train and Validation Scores showing Bad Generalization</font>**

#### Observations
At this point, it is worth noting that our model features consist of just the Aggregate Features and the Aggregate Data over Time Interval Features. Looking at these scores we can see that our model is slightly overfitting but what is most concerning is our extremely low precision score compared to recall. To understand this effect we need to look at our Validation Model's confusion matrix. Here we see that False Positives from our models are WAY too high in comparison to our False Negatives. However when we take into consideration our use case, if KKBox were going off such a model they might be losing more on treatment costs than on actual Churn haha. No substantial difference between GBT and RFC in terms of validation scores, however, RFC seemed to generalize best.

#### Notes for Next Iteration
The first thing we need to address are the False Positive quantities in both models. As our model is currently more sensitive towards making a Churn prediction, I believe that by introducing more Non-Churn data we might be able to dial down that sensitivity. Specifically, we will create 7 total subsets, 1-to-1 through 13-to-1 (Non-Churn to Churn) on every odd interval, as we hope to improve the balance between False Positives and False Negatives. Secondly, we will be adding more trend-focused features to improve the overall performance of our model. 

### Model #2: Multiple Ratio Subset Splits With Trend Features

- **<font color=orange>Train and Validation Scores and Differences</font>**

#### Observations
We now have ~230 Features with the inclusion of our *Bi-Weekly Activity Block* and *Comparison of Bi-Weekly Activity Block Features*. Looking at the results above we can see that we have made some improvements to AUC. We also notice how Recall increased significantly over the Ratio'd subsets this means that subsets seemed to have helped address the issue of having a high amount of False Positives.

We can see that our models are still overfitting with the higher ratio models being the worst. However, our Precision scores are better over our higher ratio models along with an improvement in model error and a slight decrease in Recall. Each of these models have their pros and cons and we might benefit from some sort of ensemble of these models. Before we do such a thing let's play around a bit with feature selection to see if we improve on the overall generalization between our Train and Validation models.

#### Notes for Next Iteration
Due to the lack of consistency in generalization, we will create various models based on feature importance at different thresholds. To do this we will first derive the average of all feature importance scores across all models. Then we will produce a 5 Number Summary on these scores and group our features based on their scores against the following thresholds: Mean, 75th Percentile, 50th Percentile, and 25th Percentile. Lastly, we will conduct hyperparameterization and cross-validation a bit more thoroughly. Specifically, we will increase the number of folds and adjust the Max Tree depth to reduce the complexity of our model.

### Model #3: Multiple Ratio Subset Splits @ Adjusted Thresholds 

- **<font color=orange>Feautre Importance Thresholds for GBT and RFC</font>**

- **<font color=orange>CMs for GBT and RFC - Train and Valid</font>**
- **<font color=orange>Best Model Scores for GBT and RFC - Train and Valid</font>**
- **<font color=orange> GBT and RFC Score Differences</font>**

#### Observations
To simplify our analysis we first selected the models for each threshold which generalized best. Through the combo of feature selection based on importance scores and hyperparameterization/cross-validation, we were able to increase the overall generalization of all our models. With this, we have an optimal foundation to create an ensemble model of all the best models.

#### Notes for Next Iteration
As we stated in Model #2, all of these models both have their pros and cons. So in an attempt to create the best performing model, we will create an ensemble of all models. To do this we will create a data frame of all models and their respective predictions for each member. We will conduct Recursive Feature Elimination with an XGBoost estimator.

### Model #4:

- **<font color=orange>Feautre Importance Thresholds for GBT and RFC</font>**

- **<font color=orange>Best Model Scores for GBT and RFC - Train and Valid</font>**
- **<font color=orange>CMs for GBT and RFC - Train and Valid</font>**

- **<font color=orange> GBT and RFC Score Differences</font>**

#### Observations




- Then speak to how we are going to make an optimal model from the best of each of estimator. 



# 5. Evaluation
- ***Evaluation:*** in this stage, we assess the degree to which the model meets our business objectives and seek to determine if there is some business reason why this model is deficient


### 4.3 Model Testing

In [1]:
I select which ever model has the best balance of generalization and Precision

SyntaxError: invalid syntax (<ipython-input-1-709b65810488>, line 1)

### 4.4 Results Summary